# 1 - Objetivo

A intenção deste exercício é observar como realizar avaliação de modelos de machine learning utilizando divisão do dataset (em treino, teste e validação) e utilizando *cross-validation*. Neste primeiro momento nós vamos utilizar funções padrão da biblioteca scikit-learn para estudar estes algoritmos  de machine learning. 

A [documentação do scikit-learn](https://scikit-learn.org/stable/modules/classes.html) é abrangente e possui diversos links com a parte teórica dos algoritmos, tutoriais e exemplos práticos.

## 2 - Carregando as bibliotecas

Scikit-learn possui uma interface limpa e intuitiva e todos os componentes da biblioteca expõem a mesma interface de métodos. Sendo uma biblioteca fácil de usar, tornou-se padrão na indústria de tecnologia.

Para utilizá-la, vamos primeiro carregar os métodos/módulos necessários, além de outras bibliotecas necessárias para o tutorial.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["figure.figsize"] = [6.4, 4.8]

import graphviz 
import pandas as pd
import seaborn as sns
sns.set()

from yellowbrick.style import set_palette
set_palette('yellowbrick')

# módulos do scikit-learn específicos para a avaliação:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate

## 3 - Carregando o dataset

Primeiro fazemos a preparação do `dataset` carregando-o com a biblioteca pandas:

In [2]:
data_file = 'grad_admission.csv'
dataset = pd.read_csv(data_file)

Vamos olhar alguns exemplos do `dataset` para enterndermos sua estrutura:

In [3]:
dataset.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research,Admit
0,337,118,4,4.5,4.5,9.65,Yes,0.92
1,324,107,4,4.0,4.5,8.87,Yes,0.76
2,316,104,3,3.0,3.5,8.00,Yes,0.72
3,322,110,3,3.5,2.5,8.67,Yes,0.80
4,314,103,2,2.0,3.0,8.21,No,0.65


## 4 - Avaliação de classificação

Vamos começar avaliando modelos de classificação.



### 4.1 - Transformação dos Dados

Como visto no tutorial passado, a implementação das árvores de decisão (e alguns outros algoritmos) na biblioteca `scikit-learn` somente aceitam a utilização de `features` numéricas. Por isso, precisamos converter as `features` catergóricas em representações numéricas.

Observe que também vamos transformar as classes do `target` pois serão necessárias mais tarde na validação cruzada.

Para isso, utilizaremos uma classe disponível na biblioteca que faz a conversão das categorias para formato numérico:

* `OrdinalEncoder` [docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder)

Para isso, nós: 

1. primeiro selecionaremos as colunas que precisam de conversão

2. criaremos um objeto `OrdinalEncoder`

3. faremos uma cópia do `dataset` original

4. transformaremos o `dataset`


*Observação:* algumas vezes, o `dataset`pode consumir muita memória RAM e, nestes casos, deve-se transformar a versão original do `dataset` e não criando a cópia.

In [4]:
grads = dataset.copy()
grads['Admit'] = (grads['Admit'] > 0.7).astype('str')

# selecionando as colunas
columns_to_tranform = ['Research']

Para facilitar a execução do resto do código, vamos separar os nomes das colunas em `features` e `target` para classificação e em `features` e `target` para regressão.

Começamos com a classificação:

In [5]:
class_features = grads.columns[0:-1]
class_features

Index(['GRE', 'TOEFL', 'Rating', 'SOP', 'LOR', 'CGPA', 'Research'], dtype='object')

In [6]:
# aqui selecionamos o último valor da lista para identificar o target
class_target = grads.columns[-1]
class_target

'Admit'

### 4.2 - Divisão em treino e teste.

Comecemos com a divisão do `dataset` em partes fixas. Neste tutorial não vamos utilizar um dataset de validação mas, caso houvesse necessidade, basta dividir criar a divisão de treino e teste e depois dividir as partes de treino novamente.

Uma das premissas do machine learning é que você não utilize os mesmos dados de treino para testar e avaliar o modelo. Ainda não vimos este assunto nas aulas (iniciaremos avaliação de modelos na 5a aula - semana que vem), mas por enquanto iremos dividir o `dataset` mesmo assim.

O scikit-learn possui uma função muito útil para nos auxiliar: `train_test_split`.

A assinatura do método com os valores `default` ([docs](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split)) é:

```{python}
x_train, x_test, y_train, y_test = train_test_split(*arrays, test_size=None, 
    train_size=None, random_state=None, suffle=True, stratify=None)
```

O retorno do método traz uma sequência ordenada de `numpy array` (ou `dataframe`, dependendo do formato dos `datasets` passados): `features` dos exemplos de treino; `features` dos exemplos de teste; `targets` dos exemplos de treino; `targets` dos exemplos de teste.

Por convenção, muitos praticantes de machine learning utilizam a letra `X` (ou `x`) para indicar a parte que contém as `features` e `Y` (ou `y`) para indicar a parte que contém os `targets`. Além disso é comum adicionar uma indicação do tipo de dataset que está sendo criado (`train`, `test` ou `valid`).

In [7]:
x_class_train, x_class_test, y_class_train, y_class_test = train_test_split(
    grads[class_features],
    grads[class_target], 
        test_size=0.33,
        random_state=42,
        shuffle=True)

In [8]:
x_class_test.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
361,334,116,4,4.0,3.5,9.54,Yes
73,314,108,4,4.5,4.0,9.04,Yes
374,315,105,2,2.0,2.5,7.65,No
155,312,109,3,3.0,3.0,8.69,No
104,326,112,3,3.5,3.0,9.05,Yes


In [9]:
# importando o OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

# criando o objeto e treinando-o no dataset
classification_encoder = OrdinalEncoder()
classification_encoder.fit(x_class_train[columns_to_tranform])


label_encoder = LabelEncoder()
label_encoder.fit(y_class_train)
y_class_train = label_encoder.transform(y_class_train)
y_class_test = label_encoder.transform(y_class_test)


# transformação dos dados
x_class_train[columns_to_tranform] = classification_encoder.transform(x_class_train[columns_to_tranform])
x_class_test[columns_to_tranform] = classification_encoder.transform(x_class_test[columns_to_tranform])

#exibindo a nova estrutura
x_class_train.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
471,311,103,3,2.0,4.0,8.09,0.0
26,322,109,5,4.5,3.5,8.80,0.0
7,308,101,2,3.0,4.0,7.90,0.0
453,319,103,3,2.5,4.0,8.76,1.0
108,331,116,5,5.0,5.0,9.38,1.0


In [10]:
x_class_test.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
361,334,116,4,4.0,3.5,9.54,1.0
73,314,108,4,4.5,4.0,9.04,1.0
374,315,105,2,2.0,2.5,7.65,0.0
155,312,109,3,3.0,3.0,8.69,0.0
104,326,112,3,3.5,3.0,9.05,1.0


In [11]:
y_class_train

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,

### 4.3 - Algoritmos

Para testar os algoritmos, vamos selecionar uma árvore de decisão e um K-Nearest Neighbors (k-NN), que é um método baseado em similaridade (e assunto da nossa próxima aula teórica).

A assinaura da árvore de decisão (`DecisionTreeClassifier`) com parâmetros `default` desta implementação é:

```{python}
DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, 
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
    max_features=None, random_state=None, max_leaf_nodes=None, 
    min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, 
    presort='deprecated', ccp_alpha=0.0)
```

A assinaura do k-NN (`KNeighborsClassifier`) com parâmetros `default` desta implementação é:


```{python}
KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', 
    leaf_size=30, p=2, metric='minkowski',  metric_params=None, n_jobs=None, 
    **kwargs)
```

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

Comecemos com a *árvore de decisão*:

In [13]:
dec_tree = DecisionTreeClassifier(
    criterion='gini', 
    splitter='best', 
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features=len(class_features), 
    random_state=42, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    class_weight=None, 
    ccp_alpha=0.0)

dec_tree.fit(x_class_train, y_class_train)

DecisionTreeClassifier(max_depth=10, max_features=7, random_state=42)

Agora o KNN:

In [14]:
knn = KNeighborsClassifier(
    n_neighbors=5, 
    weights='uniform', 
    algorithm='auto', 
    leaf_size=30, 
    p=2, 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=None)

knn.fit(x_class_train, y_class_train)

KNeighborsClassifier()

In [15]:
print("## Resultado Árvore de decisão: ##")

y_class_pred = dec_tree.predict(x_class_test)
print(confusion_matrix(y_class_test, y_class_pred))
print(classification_report(y_class_test, y_class_pred))


print("\n ## Resultado k-NN:##")
y_class_pred = knn.predict(x_class_test)
print(confusion_matrix(y_class_test, y_class_pred))
print(classification_report(y_class_test, y_class_pred))

## Resultado Árvore de decisão: ##
[[57 17]
 [16 75]]
              precision    recall  f1-score   support

           0       0.78      0.77      0.78        74
           1       0.82      0.82      0.82        91

    accuracy                           0.80       165
   macro avg       0.80      0.80      0.80       165
weighted avg       0.80      0.80      0.80       165


 ## Resultado k-NN:##
[[59 15]
 [11 80]]
              precision    recall  f1-score   support

           0       0.84      0.80      0.82        74
           1       0.84      0.88      0.86        91

    accuracy                           0.84       165
   macro avg       0.84      0.84      0.84       165
weighted avg       0.84      0.84      0.84       165



### 4.4 - validação com k-Fold

Vamos ver como funciona a validação com validação cruzada k-fold. 

In [16]:
dec_tree_kf = DecisionTreeClassifier(
    criterion='gini', 
    splitter='best', 
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features=len(class_features), 
    random_state=42, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    class_weight=None, 
    ccp_alpha=0.0)

scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

dec_tree_scores = cross_validate(dec_tree, x_class_train, y_class_train, cv=5, scoring=scoring)

knn_kf = KNeighborsClassifier(
    n_neighbors=5, 
    weights='uniform', 
    algorithm='auto', 
    leaf_size=30, 
    p=2, 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=None)

knn_scores = cross_validate(knn_kf,  x_class_train, y_class_train, cv=5, scoring=scoring)



In [17]:
print('precision \t{:.2f}'.format(dec_tree_scores['test_precision'].mean()))
print('recall \t\t{:.2f}'.format(dec_tree_scores['test_recall'].mean()))
print('f1-score \t{:.2f}\n'.format(dec_tree_scores['test_f1'].mean()))

print(dec_tree_scores)

print('\n\nprecision \t{:.2f}'.format(knn_scores['test_precision'].mean()))
print('recall \t\t{:.2f}'.format(knn_scores['test_recall'].mean()))
print('f1-score \t{:.2f}\n'.format(knn_scores['test_f1'].mean()))

print(knn_scores)

precision 	0.81
recall 		0.78
f1-score 	0.79

{'fit_time': array([0.0033524 , 0.00275445, 0.00269127, 0.00271082, 0.00298166]), 'score_time': array([0.00361109, 0.00383162, 0.00339341, 0.00380778, 0.00354743]), 'test_precision': array([0.81818182, 0.83783784, 0.8       , 0.8       , 0.79069767]), 'test_recall': array([0.69230769, 0.79487179, 0.71794872, 0.82051282, 0.85      ]), 'test_f1': array([0.75      , 0.81578947, 0.75675676, 0.81012658, 0.81927711])}


precision 	0.85
recall 		0.85
f1-score 	0.85

{'fit_time': array([0.00235343, 0.00288892, 0.00236058, 0.00230312, 0.00268316]), 'score_time': array([0.00611067, 0.00629926, 0.00598407, 0.0059514 , 0.0080142 ]), 'test_precision': array([0.91428571, 0.83333333, 0.82926829, 0.83333333, 0.8372093 ]), 'test_recall': array([0.82051282, 0.76923077, 0.87179487, 0.8974359 , 0.9       ]), 'test_f1': array([0.86486486, 0.8       , 0.85      , 0.86419753, 0.86746988])}


## 5 - Avaliação de regressão

Vamos começar avaliando modelos de regressão.

### 5.1 - Transformação dos Dados

Como visto anteriormente, a implementação de alguns outros algoritmos na biblioteca `scikit-learn` somente aceitam a utilização de `features` numéricas. Por isso, precisamos converter as `features` catergóricas em representações numéricas.

Para isso, utilizaremos mais uma vez o `OrdinalEncoder` [docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder).

### Dataset de regressão


Vamos modificar nosso `dataset` para que possamos reutilizá-lo para um problema de regressão. Para isso, vamos utilizar a coluna `balance`, que é numérica, como alvo e incluir a coluna `y` como uma `feature` normal. 

Note que termos que fazer a transformação da coluna `y` para que ela seja numérica como feito anteriormente com as demais. Após a transformação, vamos aproveitar e reordenar as colunas do `dataset`.

In [18]:
# selecionando as colunas
columns_to_tranform = ['Research']

# importando o OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder

# criando o objeto e treinando-o no dataset
regression_encoder = OrdinalEncoder()
regression_encoder.fit(dataset[columns_to_tranform])

# criando a cópia do dataset
regression_dataset = dataset.copy()

# transformação dos dados
regression_dataset[columns_to_tranform] = regression_encoder.transform(dataset[columns_to_tranform])


#exibindo a nova estrutura
regression_dataset.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research,Admit
0,337,118,4,4.5,4.5,9.65,1.0,0.92
1,324,107,4,4.0,4.5,8.87,1.0,0.76
2,316,104,3,3.0,3.5,8.00,1.0,0.72
3,322,110,3,3.5,2.5,8.67,1.0,0.80
4,314,103,2,2.0,3.0,8.21,0.0,0.65


In [19]:
reg_features = regression_dataset.columns[0:-1]
reg_features

Index(['GRE', 'TOEFL', 'Rating', 'SOP', 'LOR', 'CGPA', 'Research'], dtype='object')

In [20]:
reg_target = regression_dataset.columns[-1]
reg_target

'Admit'

### 5.2 - Divisão em treino e teste

Vamos começar com a divisão em treino e teste para a regressão utilizando `train_test_split` outra vez.

In [21]:
x_reg_train, x_reg_test, y_reg_train, y_reg_test = train_test_split(
    regression_dataset[reg_features],
    regression_dataset[reg_target], 
        test_size=0.33,
        random_state=42,
        shuffle=True)

In [22]:
x_reg_train.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
471,311,103,3,2.0,4.0,8.09,0.0
26,322,109,5,4.5,3.5,8.80,0.0
7,308,101,2,3.0,4.0,7.90,0.0
453,319,103,3,2.5,4.0,8.76,1.0
108,331,116,5,5.0,5.0,9.38,1.0


In [23]:
x_reg_test.head()

,GRE,TOEFL,Rating,SOP,LOR,CGPA,Research
361,334,116,4,4.0,3.5,9.54,1.0
73,314,108,4,4.5,4.0,9.04,1.0
374,315,105,2,2.0,2.5,7.65,0.0
155,312,109,3,3.0,3.0,8.69,0.0
104,326,112,3,3.5,3.0,9.05,1.0


### 5.3 - Algoritmos

Para demonstrar e regressão, vamos utilizar apenas uma regressão logística, que é um método baseado em erro e vamos estudá-lo próximo ao final da disciplina.

A assinaura da regressão logística (`LogisticRegression`) com parâmetros `default` desta implementação é:

```{python}
LinearRegression(*, fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
```

In [24]:
from sklearn.linear_model import LinearRegression

In [25]:
lin_reg = LinearRegression(
    fit_intercept=True, 
    normalize=False, 
    copy_X=True, 
    n_jobs=-1)

lin_reg.fit(x_reg_train, y_reg_train)

y_reg_pred = lin_reg.predict(x_reg_test)

print("Mean squared error: \t{:.4f}".format(mean_squared_error(y_reg_test, y_reg_pred)))
print("R2 score: \t\t{:.4f}".format(r2_score(y_reg_test, y_reg_pred)))

Mean squared error: 	0.0036
R2 score: 		0.8242


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


### 5.4 - validação com k-Fold

Agora vamos ver como funciona a validação com validação cruzada k-fold. 

In [28]:
x_reg = regression_dataset[reg_features]
y_reg = regression_dataset[reg_target]


lin_reg_kf = LinearRegression(
    fit_intercept=True, 
    normalize=False, 
    copy_X=True, 
    n_jobs=-1)

scoring = {
    'mean_squared_error': 'neg_mean_squared_error',
    'r2_score': 'r2'
}

lin_reg_scores = cross_validate(lin_reg_kf, x_reg, y_reg, cv=5, scoring=scoring)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave 

In [27]:
print('mean squared error \t{:.2f}'.format(lin_reg_scores['test_mean_squared_error'].mean() * -1.0))
print('r2-score \t\t{:.2f}\n\n'.format(lin_reg_scores['test_r2_score'].mean()))

lin_reg_scores

mean squared error 	0.00
r2-score 		0.81




{'fit_time': array([0.00904512, 0.00557613, 0.00535774, 0.00445676, 0.01306224]),
 'score_time': array([0.00458312, 0.00417471, 0.00335002, 0.01627469, 0.00405526]),
 'test_mean_squared_error': array([-0.00980581, -0.00309019, -0.00169459, -0.00343832, -0.00184317]),
 'test_r2_score': array([0.67763918, 0.79424809, 0.86447645, 0.81935698, 0.89828691])}